In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from os import listdir
from tqdm import tqdm
from os import stat, mkdir

In [2]:
path = 'C:/Users/Marie/Desktop/MSc DSBA/4. Machine Learning/3. Project'
directory = '/data_cleaned'

In [3]:
try :
    stat(path + directory)
except :
    mkdir(path + directory)

In [4]:
print(listdir(path))

['.ipynb_checkpoints', 'data_cleaned', 'Dealing with sessions.ipynb', 'Github', 'sessions.csv', 'sessions_action.csv', 'sessions_action_detail.csv', 'sessions_action_type.csv', 'sessions_cleaned', 'sessions_device_type.csv', 'sessions_secs_elapsed.csv', 'test_users.csv', 'train_users_2.csv']


In [5]:
def find_most_common_values(data, proportion = .9, col = 'action'):
    
    count = data[col].value_counts(normalize=True, dropna = False)
    count_cum = count.cumsum()
    
    df = pd.DataFrame(count)
    df[col + '_cum'] = count_cum
    
    return df[df[col+'_cum']<=proportion]
        

### Dealing with action variable

In [6]:
sessions_action = pd.read_csv(path + '/sessions.csv', usecols = ['action'])

In [7]:
df_action = find_most_common_values(data = sessions_action, proportion = .95).reset_index()

In [8]:
most_common_values = np.array(df_action.sort_values('action', ascending = False)['index'])
print(most_common_values)
del df_action, sessions_action

['show' 'index' 'search_results' 'personalize' 'search'
 'ajax_refresh_subtotal' 'update' 'similar_listings' 'social_connections'
 'reviews' 'active' 'similar_listings_v2' 'lookup' 'create' 'dashboard'
 'header_userpic' 'collections' 'edit' 'campaigns' 'track_page_view' nan
 'unavailabilities' 'qt2' 'notifications' 'confirm_email' 'requested'
 'identity' 'ajax_check_dates' 'show_personalize' 'ask_question' 'listings'
 'authenticate' 'calendar_tab_inner2' 'travel_plans_current'
 'edit_verification' 'ajax_lwlb_contact' 'other_hosting_reviews_first'
 'recommendations' 'impressions' 'manage_listing' 'click' 'complete_status'
 'ajax_photo_widget_form_iframe' 'payment_instruments'
 'message_to_host_focus' 'verify' 'payment_methods' 'cancellation_policies'
 'callback' 'settings' 'custom_recommended_destinations']


In [9]:
session_reader = pd.read_csv(path + '/sessions.csv', chunksize = 20000)
dfs = []
for chunk in tqdm(session_reader):
    chunk['action'] = chunk['action'].apply(lambda x: str(x) if str(x) in most_common_values else 'other')
    chunk = chunk[['action', 'user_id']]
    chunk = pd.get_dummies(chunk, columns = ['action'], prefix = 'action_')
    chunk = chunk.groupby('user_id').sum()
    dfs.append(chunk)

529it [01:44,  5.07it/s]


In [10]:
df = dfs[0]
for i in tqdm(range(1, len(dfs))):
    df = pd.concat([df, dfs[i]])
    dfs[i]=False

100%|████████████████████████████████████████████████████████████████████████████████| 528/528 [00:19<00:00, 27.10it/s]


In [11]:
del dfs

In [12]:
df = df.fillna(0).astype(int).reset_index()

In [13]:
df = df.groupby('user_id').sum()

In [14]:
df.head()

,action__active,action__ajax_check_dates,action__ajax_lwlb_contact,action__ajax_photo_widget_form_iframe,action__ajax_refresh_subtotal,action__ask_question,action__authenticate,action__calendar_tab_inner2,action__callback,action__campaigns,...,action__show,action__show_personalize,action__similar_listings,action__similar_listings_v2,action__social_connections,action__track_page_view,action__travel_plans_current,action__unavailabilities,action__update,action__verify
user_id,,,,,,,,,,,,,,,,,,,,,
00023iyk9l,0,0,0,0,2,0,0,0,1,0,...,9,0,3,0,0,0,2,0,0,0
0010k6l0om,0,0,0,0,8,0,0,0,1,0,...,20,0,0,8,0,0,0,0,0,0
001wyh0pz8,4,0,0,0,0,0,0,0,0,3,...,6,0,0,0,2,0,0,0,1,0
0028jgx1x1,1,0,0,0,0,0,0,0,0,0,...,15,0,0,0,0,0,0,0,0,0
002qnbzfs5,29,0,0,0,0,0,0,0,0,28,...,232,0,0,0,72,0,0,21,13,0


In [15]:
df.to_csv(path + directory + '/sessions_action.csv', index = True)

In [16]:
del df 

### Dealing with action_type variable

In [17]:
sessions_action_type = pd.read_csv(path + '/sessions.csv', usecols = ['action_type'])

In [18]:
df_action_type = find_most_common_values(proportion = .99, data = sessions_action_type, col = 'action_type').reset_index()

In [19]:
most_common_values = np.array(df_action_type.sort_values('action_type', ascending = False)['index'])
print(most_common_values)
del df_action_type, sessions_action_type

['view' 'data' 'click' nan '-unknown-' 'submit']


In [20]:
session_reader = pd.read_csv(path + '/sessions.csv', chunksize = 20000)
dfs = []
for chunk in tqdm(session_reader):
    chunk['action_type'] = chunk['action_type'].apply(lambda x: str(x) if str(x) in most_common_values else 'other')
    chunk = chunk[['action_type', 'user_id']]
    chunk = pd.get_dummies(chunk, columns = ['action_type'], prefix = 'action_type_')
    chunk = chunk.groupby('user_id').sum()
    dfs.append(chunk)

529it [01:22,  6.44it/s]


In [21]:
df = dfs[0]
for i in tqdm(range(1, len(dfs))):
    df = pd.concat([df, dfs[i]])
    dfs[i]=False

100%|███████████████████████████████████████████████████████████████████████████████| 528/528 [00:03<00:00, 145.20it/s]


In [22]:
del dfs

In [23]:
df = df.fillna(0).astype(int).reset_index()

In [24]:
df = df.groupby('user_id').sum()

In [25]:
df.head()

,action_type__-unknown-,action_type__click,action_type__data,action_type__other,action_type__submit,action_type__view
user_id,,,,,,
00023iyk9l,0,4,9,6,0,21
0010k6l0om,5,16,9,16,0,17
001wyh0pz8,6,66,2,5,3,8
0028jgx1x1,1,9,5,0,1,15
002qnbzfs5,184,140,140,94,15,216


In [26]:
df.to_csv(path + directory + '/sessions_action_type.csv', index = True)

In [27]:
del df

### Dealing with action_detail variable

In [28]:
sessions_action_detail = pd.read_csv(path + '/sessions.csv', usecols = ['action_detail'])

In [29]:
df_action_detail = find_most_common_values(proportion = .99, data = sessions_action_detail, col = 'action_detail').reset_index()

In [30]:
most_common_values = np.array(df_action_detail.sort_values('action_detail', ascending = False)['index'])
print(most_common_values)
del df_action_detail, sessions_action_detail

['view_search_results' 'p3' nan '-unknown-' 'wishlist_content_update'
 'user_profile' 'change_trip_characteristics' 'similar_listings'
 'user_social_connections' 'update_listing' 'listing_reviews' 'dashboard'
 'user_wishlists' 'header_userpic' 'message_thread' 'edit_profile'
 'message_post' 'contact_host' 'unavailable_dates' 'confirm_email_link'
 'create_user' 'change_contact_host_dates' 'user_profile_content_update'
 'user_reviews' 'p5' 'login' 'your_trips' 'p1' 'notifications'
 'profile_verifications' 'reservations' 'user_listings' 'your_listings'
 'listing_recommendations' 'update_user' 'create_phone_numbers' 'p4'
 'update_listing_description' 'update_user_profile' 'manage_listing'
 'payment_instruments' 'account_notification_settings'
 'message_to_host_focus' 'signup' 'cancellation_policies' 'oauth_response'
 'message_inbox' 'view_listing' 'message_to_host_change' 'list_your_space'
 'pending' 'wishlist' 'profile_references' 'apply_coupon' 'oauth_login'
 'view_reservations' 'login_p

In [31]:
session_reader = pd.read_csv(path + '/sessions.csv', chunksize = 20000)
dfs = []
for chunk in tqdm(session_reader):
    chunk['action_detail'] = chunk['action_detail'].apply(lambda x: str(x) if str(x) in most_common_values else 'other')
    chunk = chunk[['action_detail', 'user_id']]
    chunk = pd.get_dummies(chunk, columns = ['action_detail'], prefix = 'action_detail_')
    chunk = chunk.groupby('user_id').sum()
    dfs.append(chunk)

529it [01:44,  5.05it/s]


In [32]:
df = dfs[0]
for i in tqdm(range(1, len(dfs))):
    df = pd.concat([df, dfs[i]])
    dfs[i]=False

100%|████████████████████████████████████████████████████████████████████████████████| 528/528 [00:23<00:00, 22.10it/s]


In [33]:
del dfs

In [34]:
df = df.fillna(0).astype(int).reset_index()

In [35]:
df = df.groupby('user_id').sum()

In [36]:
df.head()

,action_detail__-unknown-,action_detail__account_notification_settings,action_detail__apply_coupon,action_detail__book_it,action_detail__cancellation_policies,action_detail__change_contact_host_dates,action_detail__change_trip_characteristics,action_detail__confirm_email_link,action_detail__contact_host,action_detail__create_listing,...,action_detail__user_reviews,action_detail__user_social_connections,action_detail__user_wishlists,action_detail__view_listing,action_detail__view_reservations,action_detail__view_search_results,action_detail__wishlist,action_detail__wishlist_content_update,action_detail__your_listings,action_detail__your_trips
user_id,,,,,,,,,,,,,,,,,,,,,
00023iyk9l,0,0,0,0,0,0,2,1,0,0,...,0,0,0,0,0,5,0,4,0,2
0010k6l0om,5,0,0,0,0,0,8,0,0,0,...,0,0,0,0,0,10,0,8,0,0
001wyh0pz8,6,0,0,0,0,0,0,0,0,0,...,0,2,4,0,0,66,0,0,0,0
0028jgx1x1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,9,0,0,0,0
002qnbzfs5,184,0,0,6,0,0,0,1,8,0,...,3,72,0,0,0,125,0,0,0,0


In [37]:
df.to_csv(path + directory + '/sessions_action_detail.csv', index = True)

In [38]:
del df

### Dealing with other variables 

In [39]:
df = pd.read_csv(path + '/sessions.csv', usecols=['user_id', 'device_type'])

In [40]:
df = pd.get_dummies(df, columns=['device_type'], prefix='decive_type_')

In [41]:
df = df.groupby('user_id').sum().astype(int)

In [42]:
df.to_csv(path + directory + '/sessions_device_type.csv', index = True)

### Dealing with continuous variables 

In [43]:
df = pd.read_csv(path + '/sessions.csv', usecols = ['user_id', 'secs_elapsed'])

In [44]:
df['aggregator'] = 1

In [45]:
df = df.groupby('user_id').agg({'secs_elapsed':[np.min, np.max, np.median, np.std, np.mean], 'aggregator':np.sum})
df.columns = ['secs_min', 'secs_max', 'secs_med', 'secs_std', 'secs_mean', 'nb_sessions']

In [46]:
df.secs_std = df.secs_std.fillna(0)

In [47]:
df.to_csv(path + directory + '/sessions_secs_elapsed.csv', index = True)

In [48]:
del df

### Concatenating different datasets

In [49]:
print(listdir(path + directory))

['sessions_action.csv', 'sessions_action_detail.csv', 'sessions_action_type.csv', 'sessions_device_type.csv', 'sessions_secs_elapsed.csv']


In [50]:
dfs = []
for f in listdir(path + directory):
    dfs.append(pd.read_csv(path + directory + '/' + f).set_index('user_id', drop = True))

In [51]:
data = pd.concat(dfs, axis = 1)

In [52]:
del dfs

In [53]:
data.to_csv(path + directory + '/full_cleaned_sessions_data.csv', index = True)

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135483 entries, 00023iyk9l to zzzlylp57e
Columns: 141 entries, action__active to nb_sessions
dtypes: float64(5), int64(136)
memory usage: 146.8+ MB


In [55]:
data.head()

,action__active,action__ajax_check_dates,action__ajax_lwlb_contact,action__ajax_photo_widget_form_iframe,action__ajax_refresh_subtotal,action__ask_question,action__authenticate,action__calendar_tab_inner2,action__callback,action__campaigns,...,decive_type__Windows Phone,decive_type__iPad Tablet,decive_type__iPhone,decive_type__iPodtouch,secs_min,secs_max,secs_med,secs_std,secs_mean,nb_sessions
user_id,,,,,,,,,,,,,,,,,,,,,
00023iyk9l,0,0,0,0,2,0,0,0,1,0,...,0,0,4,0,0.0,567165.0,850.0,92242.561900,22253.743590,40
0010k6l0om,0,0,0,0,8,0,0,0,1,0,...,0,0,0,0,3.0,127898.0,948.5,22751.227918,9460.370968,63
001wyh0pz8,4,0,0,0,0,0,0,0,0,3,...,0,0,0,0,35.0,50548.0,1105.0,6569.648182,3179.382022,90
0028jgx1x1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3.0,84636.0,1614.0,18004.964337,9900.333333,31
002qnbzfs5,29,0,0,0,0,0,0,0,0,28,...,0,0,775,0,0.0,1418284.0,498.0,58110.646170,8232.335025,789
